In [1]:
import time
import csv
import itertools
import logging
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
import tensorflow as tf

from sage.all import *
from sage.numerical.mip import MIPSolverException

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.metrics import ConfusionMatrixDisplay


2025-03-06 15:01:59.983380: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 15:01:59.987072: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-06 15:01:59.989073: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-06 15:01:59.994022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741269720.002059 2047013 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741269720.00

In [2]:
# Configure structured logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Set a fixed random seed for reproducibility
np.random.seed(42)

In [3]:
##################################
# Data Loading
##################################

def load_mnist_train_and_test():
    """
    Loads the official MNIST training (60,000 samples) and test (10,000 samples) sets
    from tf.keras.datasets.mnist. The images are flattened into 784-dimensional vectors.
    """
    logging.info("Loading MNIST dataset from tf.keras.datasets...")
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape(-1, 28*28).astype(np.float64)
    X_test = X_test.reshape(-1, 28*28).astype(np.float64)
    logging.info(f"Official training set: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")
    logging.info(f"Official test set: X_test shape = {X_test.shape}, y_test shape = {y_test.shape}")
    return X_train, y_train, X_test, y_test


# # Global dataset variables
# X_full_global, y_full_global = load_mnist()

In [4]:
##################################
# Preprocessing & Feature Generation
##################################
def preprocess_data(X_train, X_test, n_components=15):
    """
    Scales training and test data and applies PCA based on the training data.
    Returns the PCA-transformed data along with the scaler and PCA objects.
    """
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    return X_train_pca, X_test_pca, scaler, pca

In [5]:
def create_polynomial_features(X, degree):
    """
    Creates polynomial features using scikit‑learn’s PolynomialFeatures.
    """
    poly = PolynomialFeatures(degree=degree, include_bias=True)
    return poly.fit_transform(X), poly

In [6]:
##################################
# Optimization Functions (Using Sage/GLPK)
##################################
# def is_feasible_for_z(p_Phi, q_Phi, f_c, z_current):
#     """
#     Check feasibility of the rational approximation for a fixed z_current.
#     Uses different polynomial expansions for numerator and denominator.
#     """
#     N = p_Phi.shape[0]
#     p_num_terms = p_Phi.shape[1]
#     q_num_terms = q_Phi.shape[1]
#     p = MixedIntegerLinearProgram(maximization=False, solver="GLPK")
#     p_vars = p.new_variable(real=True, name="pvars", indices=range(p_num_terms))
#     q_vars = p.new_variable(real=True, name="qvars", indices=range(q_num_terms))
#     # Fix q_0 = 1 to avoid trivial solutions
#     p.add_constraint(q_vars[0] == 1)
#     for i in range(N):
#         p_xi = sum(p_Phi[i, j] * p_vars[j] for j in range(p_num_terms))
#         q_xi = sum(q_Phi[i, k] * q_vars[k] for k in range(q_num_terms))
#         p.add_constraint(f_c[i]*q_xi - p_xi - z_current*q_xi <= 0)
#         p.add_constraint(p_xi - f_c[i]*q_xi - z_current*q_xi <= 0)
#     p.set_objective(0)
#     try:
#         p.solve()
#         p_coeffs = np.array([p.get_values(p_vars[j]) for j in range(p_num_terms)])
#         q_coeffs = np.array([p.get_values(q_vars[k]) for k in range(q_num_terms)])
#         return True, p_coeffs, q_coeffs
#     except MIPSolverException:
#         return False, None, None


In [7]:
##################################
# Optimization Functions (Using Gurobi)
##################################
from gurobipy import Model, GRB

def is_feasible_for_z(p_Phi, q_Phi, f_c, z_current):
    """
    Uses Gurobi optimizer to check feasibility of the rational approximation for a fixed z_current.
    p_Phi: numpy array for numerator polynomial features (training data)
    q_Phi: numpy array for denominator polynomial features (training data)
    f_c: binary indicator vector for the class (length = number of training samples)
    z_current: the current error bound
    Returns (feasible, p_coeffs, q_coeffs)
    """
    N = p_Phi.shape[0]
    p_num_terms = p_Phi.shape[1]
    q_num_terms = q_Phi.shape[1]
    
    # Create Gurobi model
    model = Model("rational_approx")
    model.setParam("OutputFlag", 0)  # suppress output
    
    # Create variables: one set for numerator coefficients and one for denominator coefficients
    p_vars = model.addVars(p_num_terms, lb=-GRB.INFINITY, name="p")
    q_vars = model.addVars(q_num_terms, lb=-GRB.INFINITY, name="q")
    
    # Fix q_0 to avoid trivial solutions
    model.addConstr(q_vars[0] ==0.0001, "fix_q0")

    # Set a small threshold to ensure q(x_i) stays positive
    delta = 1e-6
    
    # For each training sample, add two constraints:
    # 1. f(x_i)*q(x_i) - p(x_i) <= z_current * q(x_i)
    # 2. p(x_i) - f(x_i)*q(x_i) <= z_current * q(x_i)
    for i in range(N):
        # Compute p(x_i) and q(x_i)
        p_xi = sum(p_Phi[i, j] * p_vars[j] for j in range(p_num_terms))
        q_xi = sum(q_Phi[i, k] * q_vars[k] for k in range(q_num_terms))
        
        model.addConstr(f_c[i] * q_xi - p_xi <= z_current * q_xi, name=f"c1_{i}")
        model.addConstr(p_xi - f_c[i] * q_xi <= z_current * q_xi, name=f"c2_{i}")
        # Constraint 3: Enforce that q(x_i) >= delta
        model.addConstr(q_xi >= delta, name=f"positivity_{i}")
    
    # No objective (feasibility check) so we simply set an objective of zero.
    model.setObjective(0, GRB.MINIMIZE)
    
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        p_coeffs = np.array([p_vars[j].X for j in range(p_num_terms)])
        q_coeffs = np.array([q_vars[k].X for k in range(q_num_terms)])
        return True, p_coeffs, q_coeffs
    else:
        return False, None, None

In [9]:
def solve_rational_approx_bisection(p_Phi, q_Phi, f_c, z_init_high=10.0, tolerance=1e-8):
    """
    Uses a bisection method on z to find a feasible rational approximation.
    """
    z_low = 0.0
    z_high = z_init_high
    best_p, best_q = None, None
    while (z_high - z_low) >= tolerance:
        z_current = 0.5 * (z_low + z_high)
        feasible, p_coeffs, q_coeffs = is_feasible_for_z(p_Phi, q_Phi, f_c, z_current)
        if feasible:
            z_high = z_current
            best_p = p_coeffs
            best_q = q_coeffs
        else:
            z_low = z_current
    if best_p is None:
        return None, None, None
    z_approx = 0.5 * (z_low + z_high)
    return best_p, best_q, z_approx


In [10]:
def solve_for_class(c, p_Phi_train, q_Phi_train, f_values):
    """
    Solves the rational approximation for class c.
    Returns (c, p_coeffs, q_coeffs, z_approx).
    """
    logging.info(f"Solving for digit {c}...")
    p_coeffs, q_coeffs, z_approx = solve_rational_approx_bisection(p_Phi_train, q_Phi_train, f_values[c], z_init_high=10.0)
    if p_coeffs is None:
        logging.info(f"Digit {c}: No feasible solution found.")
        return (c, None, None, None)
    else:
        logging.info(f"Digit {c}: z_approx = {z_approx}")
        return (c, p_coeffs, q_coeffs, z_approx)
        
def evaluate_rational(p_coeffs, q_coeffs, x_pca, p_poly, q_poly):
    """
    Evaluates p(x)/q(x) for a single sample.
    """
    x_pca_reshaped = x_pca.reshape(1, -1)
    p_feat = p_poly.transform(x_pca_reshaped)[0]
    q_feat = q_poly.transform(x_pca_reshaped)[0]
    p_val = np.dot(p_coeffs, p_feat)
    q_val = np.dot(q_coeffs, q_feat)
    if abs(q_val) < 1e-12:
        return np.inf if p_val > 0 else -np.inf
    return p_val / q_val

In [11]:
##################################
# Experiment Function
##################################
def run_experiment(N_samples, num_degree, den_degree, n_pca_components, subset_test_size=None):
    """
    Runs one experiment using the official Keras train/test split.
    """
    if subset_test_size is None:
        subset_test_size = int(0.2 * N_samples)
        
    X_train_full, y_train_full, X_test_full, y_test_full = load_mnist_train_and_test()

    # Sample N_samples from the training set
    indices = np.random.choice(X_train_full.shape[0], N_samples, replace=False)
    X_train_subset = X_train_full[indices]
    y_train_subset = np.array(y_train_full)[indices]
    logging.info(f"Experiment with N_samples={N_samples}, num_degree={num_degree}, den_degree={den_degree}, n_pca_components={n_pca_components}")
    logging.info(f"Sampled training dataset shapes: {X_train_subset.shape}, {y_train_subset.shape}")
    
    # Preprocess: scale and apply PCA. Use training subset to fit scaler and PCA, then transform the entire test set.
    X_train_pca, X_test_pca, scaler, pca = preprocess_data(X_train_subset, X_test_full, n_components=n_pca_components)

    # # Scaling and PCA on training data; apply same transformation to test data.
    # scaler = StandardScaler()
    # X_train_scaled = scaler.fit_transform(X_train)
    # X_test_scaled = scaler.transform(X_test)
    
    # pca = PCA(n_pca_components)
    # X_train_pca = pca.fit_transform(X_train_scaled)
    # X_test_pca = pca.transform(X_test_scaled)

    pca = PCA(n_components=n_pca_components)
    pca.fit(X_train_subset)
    
    # Get the singular values
    singular_values = pca.singular_values_
    # Enumerate starting from 1 and format each pair
    pairs = "".join(f"({i+1},{value:.2f})" for i, value in enumerate(singular_values))
    print(pairs)

    
    # Create polynomial features with separate degrees
    p_poly = PolynomialFeatures(degree=num_degree, include_bias=True)
    p_Phi_train = p_poly.fit_transform(X_train_pca)
    p_Phi_test  = p_poly.transform(X_test_pca)
    
    q_poly = PolynomialFeatures(degree=den_degree, include_bias=True)
    q_Phi_train = q_poly.fit_transform(X_train_pca)
    q_Phi_test  = q_poly.transform(X_test_pca)
    
    # Create binary labels for each class from the training set
    classes = range(10)
    f_values = {c: np.array([1 if label == c else 0 for label in y_train_subset]) for c in classes}
    
    models = {}
    start_time = time.time()
    
    # Use ProcessPoolExecutor (adjust max_workers as needed)
    solve_for_class_partial = partial(solve_for_class,
                                       p_Phi_train=p_Phi_train,
                                       q_Phi_train=q_Phi_train,
                                       f_values=f_values)
    with ProcessPoolExecutor(max_workers=32) as executor:
        results = list(executor.map(solve_for_class_partial, classes))
    
    for c, p_coeffs, q_coeffs, z_approx in results:
        models[c] = (p_coeffs, q_coeffs, z_approx) if p_coeffs is not None else None
    
    total_running_time = time.time() - start_time
    logging.info(f"Total training time (optimization phase): {total_running_time:.2f} s")
    
    # Define the classifier function using the trained models
    def classify(x):
        x_scaled = scaler.transform(x.reshape(1, -1))
        x_pca = pca.transform(x_scaled)[0]
        scores = {}
        for c, model in models.items():
            if model is None:
                scores[c] = -np.inf
            else:
                p_coeffs, q_coeffs, _ = model
                scores[c] = evaluate_rational(p_coeffs, q_coeffs, x_pca, p_poly, q_poly)
                # print(f"Digit {c}:")
                # print("  Numerator Coefficients:", p_poly)
                # print("  Denominator Coefficients:", q_poly)
        return max(scores, key=scores.get)
    
    # Evaluate on a subset (or all) of the test set
    # if subset_test_size > len(X_test):
    #     subset_test_size = len(X_test)
    # test_indices = np.random.choice(len(X_test), size=subset_test_size, replace=False)
    
    y_true = list(y_test_full)
    y_pred = []
    for i in range(len(X_test_full)):
        predicted_label = classify(X_test_full[i])
        y_pred.append(predicted_label)
    
    # Compute evaluation metrics
    test_accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred, labels=list(classes))
    class_report = classification_report(y_true, y_pred, labels=list(classes))
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    
    # Optionally display the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix (Test set: 10,000 samples)")
    plt.savefig("confusion_matrix.png", dpi=300, bbox_inches="tight")  # Save the figure
    plt.show()
    
    logging.info(f"Test accuracy on the full test set (10,000 samples): {test_accuracy}")
    logging.info(f"Classification Report:\n{class_report}")
    
    return total_running_time, test_accuracy, macro_f1, class_report


In [ ]:
##################################
# Main: Run Experiment Grid and Save Results
##################################
if __name__ == '__main__':
    sample_sizes = [784]
    num_degrees = [2]
    den_degrees = [1]
    pca_components_list = [784]
    
    csv_filename = "experiment_results3.csv"
    fieldnames = ['N_samples', 'num_degree', 'den_degree', 'n_pca_components', 'total_running_time', 'test_accuracy', 'macro_f1']
    total_experiments = len(sample_sizes) * len(num_degrees) * len(den_degrees) * len(pca_components_list)
    logging.info(f"Starting grid search with {total_experiments} experiments.")
    
    with open(csv_filename, mode='a', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for N_samples, num_degree, den_degree, n_pca_components in itertools.product(
                sample_sizes, num_degrees, den_degrees, pca_components_list):
            logging.info("-------------------------------------------------------")
            logging.info(f"Starting experiment with N_samples={N_samples}, num_degree={num_degree}, den_degree={den_degree}, n_pca_components={n_pca_components}")
            run_time, accuracy, macro_f1, report  = run_experiment(N_samples, num_degree, den_degree, n_pca_components)
            writer.writerow({
                'N_samples': N_samples,
                'num_degree': num_degree,
                'den_degree': den_degree,
                'n_pca_components': n_pca_components,
                'total_running_time': run_time,
                'test_accuracy': accuracy,
                'macro_f1': macro_f1
            })
    
    logging.info(f"All experiments finished. Results saved in {csv_filename}")

2025-03-06 15:02:05,178 - INFO - Starting grid search with 1 experiments.
2025-03-06 15:02:05,180 - INFO - -------------------------------------------------------
2025-03-06 15:02:05,180 - INFO - Starting experiment with N_samples=784, num_degree=2, den_degree=1, n_pca_components=784
2025-03-06 15:02:05,180 - INFO - Loading MNIST dataset from tf.keras.datasets...
2025-03-06 15:02:05,359 - INFO - Official training set: X_train shape = (60000, 784), y_train shape = (60000,)
2025-03-06 15:02:05,359 - INFO - Official test set: X_test shape = (10000, 784), y_test shape = (10000,)
2025-03-06 15:02:05,361 - INFO - Experiment with N_samples=784, num_degree=2, den_degree=1, n_pca_components=784
2025-03-06 15:02:05,361 - INFO - Sampled training dataset shapes: (784, 784), (784,)


(1,16907.36)(2,13152.48)(3,12620.12)(4,12182.35)(5,11836.00)(6,10623.45)(7,9177.69)(8,8643.69)(9,8423.92)(10,7914.49)(11,7562.00)(12,7279.07)(13,7202.02)(14,6914.58)(15,6554.41)(16,6384.57)(17,6116.29)(18,5988.01)(19,5708.12)(20,5521.88)(21,5345.45)(22,5237.15)(23,5120.71)(24,4971.20)(25,4867.83)(26,4709.55)(27,4637.69)(28,4606.15)(29,4513.59)(30,4475.28)(31,4344.03)(32,4282.16)(33,4219.24)(34,4028.05)(35,3983.34)(36,3951.79)(37,3832.86)(38,3773.43)(39,3676.87)(40,3627.68)(41,3488.99)(42,3416.80)(43,3410.43)(44,3302.93)(45,3289.55)(46,3259.84)(47,3160.35)(48,3148.89)(49,3043.96)(50,3010.74)(51,3005.24)(52,2917.63)(53,2906.08)(54,2855.99)(55,2835.96)(56,2716.13)(57,2679.41)(58,2637.76)(59,2622.13)(60,2592.29)(61,2568.58)(62,2543.34)(63,2483.26)(64,2455.70)(65,2433.27)(66,2387.11)(67,2355.00)(68,2345.16)(69,2297.30)(70,2284.33)(71,2263.76)(72,2239.31)(73,2194.31)(74,2180.54)(75,2174.86)(76,2151.40)(77,2110.78)(78,2075.49)(79,2040.63)(80,2026.12)(81,2005.67)(82,1992.94)(83,1953.65)(84,193

2025-03-06 15:02:14,990 - INFO - Solving for digit 0...


Set parameter Username


2025-03-06 15:02:15,003 - INFO - Set parameter Username


...

2025-03-06 15:02:17,135 - INFO - Set parameter Username


Set parameter LicenseID to value 2610482


2025-03-06 15:02:17,135 - INFO - Set parameter LicenseID to value 2610482


Academic license - for non-commercial use only - expires 2026-01-15


2025-03-06 15:02:17,137 - INFO - Academic license - for non-commercial use only - expires 2026-01-15
Process ForkProcess-7:
Process ForkProcess-20:
Process ForkProcess-31:
Process ForkProcess-27:
Process ForkProcess-14:
Process ForkProcess-19:
Process ForkProcess-32:
Process ForkProcess-21:
Process ForkProcess-13:
Process ForkProcess-11:
Process ForkProcess-9:
Process ForkProcess-8:
Process ForkProcess-5:
Process ForkProcess-12:
Process ForkProcess-16:
Process ForkProcess-23:
Process ForkProcess-22:
Process ForkProcess-17:
Process ForkProcess-6:
Process ForkProcess-18:
Process ForkProcess-25:
Process ForkProcess-15:
Process ForkProcess-30:
Process ForkProcess-26:
Process ForkProcess-3:
Process ForkProcess-10:
Process ForkProcess-24:
Process ForkProcess-4:
Traceback (most recent call last):
  File "/home/omar2425/miniforge3/envs/sage/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/omar2425/miniforge3/envs/sage/lib/python3.10/multiprocessi